## model

### BILSTM

In [1]:
import keras
import pickle
import numpy as np
import pandas as pd
from tflearn.data_utils import pad_sequences
from collections import defaultdict
from sklearn.metrics import f1_score
from keras.layers import *
from keras.models import Sequential
import tensorflow as tf
import keras.backend.tensorflow_backend as KTF
from keras import initializers
import os
import keras

from keras.optimizers import Adadelta
import gc

config = tf.ConfigProto()  
config.gpu_options.allow_growth=True   #不全部占满显存, 按需分配
sess = tf.Session(config=config)

KTF.set_session(sess)

EMBEDDING_DIM = 300
VOCAB_LENGTH = 3000

from tensorflow.python.client import device_lib


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

/home/chenchi/anaconda3/envs/py27/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
train_data_word = pd.read_csv('../data/aux/train_word_indexvec.csv')
train_data_char = pd.read_csv('../data/aux/train_char_indexvec.csv')
train_data = train_data_word
train_data.head()
nan_test = pd.isnull(train_data)
print nan_test.describe()

lda_feas_char = pd.read_csv('../lda_features_char.csv')
lda_feas_char = lda_feas_char.values
print(lda_feas_char.shape)
ngram_feas_char = pd.read_csv('../ngram_features_char.csv')
ngram_feas_char = ngram_feas_char.values
print(ngram_feas_char.shape)
simsummary_feas_char = pd.read_csv('../simsummary_features_char.csv')
simsummary_feas_char = simsummary_feas_char.values
print(simsummary_feas_char.shape)
lda_feas_word = pd.read_csv('../lda_features_word.csv')
lda_feas_word = lda_feas_word.values
print(lda_feas_word.shape)
ngram_feas_word = pd.read_csv('../ngram_features_word.csv')
ngram_feas_word = ngram_feas_word.values
print(ngram_feas_word.shape)
simsummary_feas_word = pd.read_csv('../simsummary_features_word.csv')
simsummary_feas_word = simsummary_feas_word.values
print(simsummary_feas_word.shape)
tfidf_feas = pd.read_csv('../tfidf_features.csv')
tfidf_feas = tfidf_feas.values
print(tfidf_feas.shape)

all_feas = np.concatenate([ngram_feas_char,lda_feas_char, simsummary_feas_char, tfidf_feas], axis=1)

# ngram_feas_word, lda_feas_word, simsummary_feas_word
                           
print all_feas.shape


embedding_matrix_char = np.load('../data/aux/vec_char.npy')

embedding_matrix_word = np.load('../data/aux/vec_word.npy')

            0      1      2      3
count   98976  98976  98976  98976
unique      1      1      1      1
top     False  False  False  False
freq    98976  98976  98976  98976
(98976, 2)
(98976, 16)
(98976, 5)
(98976, 2)
(98976, 16)
(98976, 5)
(98976, 4)
(98976, 27)


In [51]:

VOC_SIZE_CHAR = 2087
VOC_SIZE_WORD = 5002
from keras.layers import *
from keras.activations import softmax
from keras.models import Model

def get_embedding_output(input_x1, input_x2, params, MAX_SEQUENCE_LENGTH):
    embedding_layer = keras.layers.Embedding(
        VOCAB_LENGTH,
        EMBEDDING_DIM,
        input_length=MAX_SEQUENCE_LENGTH,
        trainable=False,
        mask_zero=True, 
    )
    embedding_seq1 = embedding_layer(input_x1)
    embedding_seq2 = embedding_layer(input_x2)
    return embedding_seq1, embedding_seq2

def get_embedding_output_vec(input_x1, input_x2, params, MAX_SEQUENCE_LENGTH, embedding_matrix, vocab_len):
    embedder = Embedding(vocab_len, 300, input_length = MAX_SEQUENCE_LENGTH, 
                         weights = [embedding_matrix], mask_zero=True, trainable = False)
    embedding_seq1 = embedder(input_x1)
    embedding_seq2 = embedder(input_x2)
    return embedding_seq1, embedding_seq2

num_stack = 3
def get_bilstm_output(input_x1, input_x2, params):
    
    lstm_layer = keras.layers.Bidirectional(keras.layers.GRU(
        params['num_lstm'],
        dropout=params['lstm_dropout_rate'],
        recurrent_dropout=params['lstm_dropout_rate'],
        return_sequences=True
    ))
    x1 = lstm_layer(input_x1)
    x2 = lstm_layer(input_x2)
    return x1, x2;

def get_cnn_output(input_x1, params):
    # cnn0模块，kernel_size = 2
    conv0_1 = Convolution1D(256, 2, padding='same')(input_x1)
    bn0_1 = BatchNormalization()(conv0_1)
    relu0_1 = Activation('relu')(bn0_1)
    conv0_2 = Convolution1D(128, 2, padding='same')(relu0_1)
    bn0_2 = BatchNormalization()(conv0_2)
    relu0_2 = Activation('relu')(bn0_2)
    cnn0 = MaxPool1D(pool_size=4)(relu0_2)
    # cnn1模块，kernel_size = 3
    conv1_1 = Convolution1D(256, 3, padding='same')(input_x1)
    bn1_1 = BatchNormalization()(conv1_1)
    relu1_1 = Activation('relu')(bn1_1)
    conv1_2 = Convolution1D(128, 3, padding='same')(relu1_1)
    bn1_2 = BatchNormalization()(conv1_2)
    relu1_2 = Activation('relu')(bn1_2)
    cnn1 = MaxPool1D(pool_size=4)(relu1_2)
    # cnn2模块，kernel_size = 4
    conv2_1 = Convolution1D(256, 4, padding='same')(input_x1)
    bn2_1 = BatchNormalization()(conv2_1)
    relu2_1 = Activation('relu')(bn2_1)
    conv2_2 = Convolution1D(128, 4, padding='same')(relu2_1)
    bn2_2 = BatchNormalization()(conv2_2)
    relu2_2 = Activation('relu')(bn2_2)
    cnn2 = MaxPool1D(pool_size=4)(relu2_2)
    # cnn3模块，kernel_size = 5
    conv3_1 = Convolution1D(256, 5, padding='same')(input_x1)
    bn3_1 = BatchNormalization()(conv3_1)
    relu3_1 = Activation('relu')(bn3_1)
    conv3_2 = Convolution1D(128, 5, padding='same')(relu3_1)
    bn3_2 = BatchNormalization()(conv3_2)
    relu3_2 = Activation('relu')(bn3_2)
    cnn3 = MaxPool1D(pool_size=4)(relu3_2)
    # 拼接三个模块
    cnn = concatenate([cnn0,cnn1,cnn2,cnn3], axis=-1)
    return cnn

class MulAttLayer(Layer):
    def __init__(self, max_seq_len, hidden_units=150, **kwargs):
        self.dense_layer = TimeDistributed(Dense(hidden_units, activation='softmax', kernel_initializer='zero'))
        self.max_seq_len = max_seq_len;
        super(MulAttLayer, self).__init__(** kwargs)
        
    def compute_mask(self, input, input_mask=None):
        return None
    
    def call(self, inputs):
        a, b, a_lengths, b_lengths = inputs
        a_norm = tf.nn.l2_normalize(a, axis=2)
        b_norm = tf.nn.l2_normalize(b, axis=2)
        logits = tf.matmul(a_norm, tf.transpose(b_norm, (0, 2, 1)))
        logits = logits - tf.expand_dims(tf.reduce_max(logits, axis=2), 2)
        attn = tf.exp(logits)
        #attn = mask_attention_weights(attn, a_lengths, b_lengths, self.max_seq_len)
        
        a_mask = tf.expand_dims(tf.sequence_mask(a_lengths, maxlen=self.max_seq_len), 2)
        b_mask = tf.expand_dims(tf.sequence_mask(b_lengths, maxlen=self.max_seq_len), 1)
        a_mask = tf.reshape(a_mask, shape=(-1, self.max_seq_len, 1))
        b_mask = tf.reshape(b_mask, shape=(-1, 1, self.max_seq_len))
        seq_mask = tf.cast(tf.matmul(tf.cast(a_mask, tf.int32), tf.cast(b_mask, tf.int32)), tf.bool)
        attn = tf.where(seq_mask, attn, tf.zeros_like(attn))
        attn = attn / tf.expand_dims(tf.reduce_sum(attn, axis=2) + 1e-10, 2)
        #attn = attention_func(a, b, a_lengths, b_lengths, max_seq_len, **attention_func_kwargs)
        return tf.reduce_max(tf.einsum('ijk,ikl->ijkl', attn, b), axis=2)
    
    def compute_output_shape(self, input_shape):
        return (input_shape[0][0], self.max_seq_len, input_shape[0][2])

class AttLayer(Layer):
    def __init__(self, init='glorot_uniform', kernel_regularizer=None, 
                 bias_regularizer=None, kernel_constraint=None, 
                 bias_constraint=None,  **kwargs):
        self.supports_masking = True
        self.init = initializers.get(init)
        self.kernel_initializer = initializers.get(init)
        
        self.kernel_regularizer = regularizers.get(kernel_regularizer)
        self.bias_regularizer = regularizers.get(kernel_regularizer)
        
        self.kernel_constraint = constraints.get(kernel_constraint)
        self.bias_constraint = constraints.get(bias_constraint)
        
        super(AttLayer, self).__init__(** kwargs)

    def build(self, input_shape):
        assert len(input_shape)==3
        self.W = self.add_weight((input_shape[-1], 1),
                                 initializer=self.kernel_initializer,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.kernel_regularizer,
                                 constraint=self.kernel_constraint)
        self.b = self.add_weight((input_shape[1],),
                                 initializer='zero',
                                 name='{}_b'.format(self.name),
                                 regularizer=self.bias_regularizer,
                                 constraint=self.bias_constraint)
        self.u = self.add_weight((input_shape[1],),
                                 initializer=self.kernel_initializer,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.kernel_regularizer,
                                 constraint=self.kernel_constraint)
        
        self.built = True
        
    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        uit = K.dot(x, self.W) # (x, 40, 1)
        uit = K.squeeze(uit, -1) # (x, 40)
        uit = uit + self.b # (x, 40) + (40,)
        uit = K.tanh(uit) # (x, 40)

        ait = uit * self.u # (x, 40) * (40, 1) => (x, 1)
        ait = K.exp(ait) # (X, 1)

        if mask is not None:
            mask = K.cast(mask, K.floatx()) #(x, 40)
            ait = mask*ait #(x, 40) * (x, 40, )

        ait /= K.cast(K.sum(ait, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        ait = K.expand_dims(ait)
        weighted_input = x * ait
        output = K.sum(weighted_input, axis=1)
        return output
    
    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])
    

### LSTM + FEATURES
def create_model(params):
    sequence_1_input_char = keras.Input(shape=(MAX_SEQUENCE_LENGTH_CHAR, ), dtype='int32')
    sequence_2_input_char = keras.Input(shape=(MAX_SEQUENCE_LENGTH_CHAR, ), dtype='int32')
    sequence_1_input_word = keras.Input(shape=(MAX_SEQUENCE_LENGTH_WORD, ), dtype='int32')
    sequence_2_input_word = keras.Input(shape=(MAX_SEQUENCE_LENGTH_WORD, ), dtype='int32')
    sequence_1_input_char_len = keras.Input(shape=(1,), dtype='int32')
    sequence_2_input_char_len = keras.Input(shape=(1,), dtype='int32')
    sequence_1_input_word_len = keras.Input(shape=(1,), dtype='int32')
    sequence_2_input_word_len = keras.Input(shape=(1,), dtype='int32')
    magic_input = keras.Input(shape=(all_feas.shape[-1], ))
    
    embedding_seq1_char, embedding_seq2_char = get_embedding_output_vec(sequence_1_input_char,
                                                          sequence_2_input_char, params, MAX_SEQUENCE_LENGTH_CHAR,
                                                                   embedding_matrix_char, 2087)
    
    embedding_seq1_word, embedding_seq2_word = get_embedding_output_vec(
        sequence_1_input_word, sequence_2_input_word, params, MAX_SEQUENCE_LENGTH_WORD, embedding_matrix_word, 5002)
    
#     x1,x2 = get_bilstm_output(, params)
    x1, x2 = embedding_seq1_char, embedding_seq2_char
    
    
    concatenate_layer = keras.layers.Concatenate()
    
    rnn_x1 = MulAttLayer(MAX_SEQUENCE_LENGTH_CHAR)([x1, x2, sequence_1_input_char_len, 
                            sequence_2_input_char_len])
    att_layer0 = keras.layers.Bidirectional(keras.layers.GRU(
        params['num_lstm'],
        dropout=params['lstm_dropout_rate'],
        recurrent_dropout=params['lstm_dropout_rate'],
        return_sequences=False, return_state=False
    ))
    rnn_x1 = att_layer0(rnn_x1)
    
    
    #x1,x2 = get_bilstm_output(embedding_seq1_word, embedding_seq2_word, params)
    x1, x2 = embedding_seq1_word, embedding_seq2_word
    rnn_x2 = MulAttLayer(MAX_SEQUENCE_LENGTH_WORD)([x1, x2, sequence_1_input_word_len, 
                            sequence_2_input_word_len])
    
    att_layer1 = keras.layers.Bidirectional(keras.layers.GRU(
        params['num_lstm'],
        dropout=params['lstm_dropout_rate'],
        recurrent_dropout=params['lstm_dropout_rate'],
        return_sequences=False, return_state=False
    ))
    rnn_x2 = att_layer1(rnn_x2)
    
    
    
    #magic_features = keras.layers.Dense(128, activation='relu')(magic_input)
    merged = concatenate_layer([rnn_x1, rnn_x2, magic_input])#
    merged = keras.layers.Dropout(params['dense_dropout_rate'])(merged)
    merged = keras.layers.BatchNormalization()(merged)

    merged = keras.layers.Dense(params['num_dense'], activation='relu')(merged)
    merged = keras.layers.Dropout(params['dense_dropout_rate'])(merged)
    merged = keras.layers.BatchNormalization()(merged)

    output = keras.layers.Dense(1, activation='sigmoid')(merged)

    model = keras.Model(
        inputs=[sequence_1_input_char, sequence_2_input_char, 
                sequence_1_input_word, sequence_2_input_word, 
                sequence_1_input_char_len, sequence_2_input_char_len,
                sequence_1_input_word_len, sequence_2_input_word_len,
                magic_input],#
        outputs=output
    )

    model.compile(
        loss='binary_crossentropy',
        optimizer='nadam',
        metrics=['accuracy']
    )
    return model

def create_model_dssm(params):
    # dssm是一个简单的双塔模型
    input1 = keras.Input(shape=(MAX_SEQUENCE_LENGTH_WORD, ), dtype='int32')
    input2 = keras.Input(shape=(MAX_SEQUENCE_LENGTH_WORD, ), dtype='int32')
    input1c = keras.Input(shape=(MAX_SEQUENCE_LENGTH_CHAR, ), dtype='int32')
    input2c = keras.Input(shape=(MAX_SEQUENCE_LENGTH_CHAR, ), dtype='int32')
    # magic_input = keras.Input(shape=(all_feas.shape[-1], ))
    
    embedder = Embedding(5002, 300, input_length = MAX_SEQUENCE_LENGTH_WORD, 
                         weights = [embedding_matrix_word])# , mask_zero=True, trainable = False
    # CuDNN
    lstm0 = LSTM(params['num_lstm'],
        dropout=params['lstm_dropout_rate'],
        recurrent_dropout=params['lstm_dropout_rate'],
                 return_sequences = True)
    lstm1 = Bidirectional(LSTM(params['num_lstm'],
        dropout=params['lstm_dropout_rate'],
        recurrent_dropout=params['lstm_dropout_rate']))
    lstm2 = LSTM(params['num_lstm'])
    # att1 = AttLayer() #Attention(10)
    den = Dense(64,activation = 'tanh')

    att1 = Lambda(lambda x: K.max(x,axis = 1))
    v1 = embedder(input1)
    v2 = embedder(input2)
    v11 = lstm1(v1)
    v22 = lstm1(v2)
    v1ls = lstm2(lstm0(v1))
    v2ls = lstm2(lstm0(v2))
    v1 = Concatenate(axis=1)([att1(v1),v11])
    v2 = Concatenate(axis=1)([att1(v2),v22])
    
    embedder = Embedding(2087, 300, input_length = MAX_SEQUENCE_LENGTH_CHAR, 
                         weights = [embedding_matrix_char])# , mask_zero=True, trainable = False
    lstm1c = Bidirectional(LSTM(params['num_lstm']))
    #att1c = Attention(10)
    att1c = Lambda(lambda x: K.max(x,axis = 1))
    # att1c = AttLayer()
    v1c = embedder(input1c)
    v2c = embedder(input2c)
    v11c = lstm1c(v1c)
    v22c = lstm1c(v2c)
    v1c = Concatenate(axis=1)([att1c(v1c),v11c])
    v2c = Concatenate(axis=1)([att1c(v2c),v22c])


    mul = Multiply()([v1,v2])
    sub = Lambda(lambda x: K.abs(x))(Subtract()([v1,v2]))
    maximum = Maximum()([Multiply()([v1,v1]),Multiply()([v2,v2])])
    mulc = Multiply()([v1c,v2c])
    subc = Lambda(lambda x: K.abs(x))(Subtract()([v1c,v2c]))
    maximumc = Maximum()([Multiply()([v1c,v1c]),Multiply()([v2c,v2c])])
    sub2 = Lambda(lambda x: K.abs(x))(Subtract()([v1ls,v2ls]))
    matchlist = Concatenate(axis=1)([mul,sub,mulc,subc,maximum,maximumc,sub2])
    matchlist = Dropout(0.2)(matchlist)
    

    merged = Concatenate(axis=1)([Dense(32,activation = 'relu')(matchlist),
                                     Dense(48,activation = 'sigmoid')(matchlist)])
    
    merged = keras.layers.Dense(params['num_dense'], activation='relu')(merged)
    # merged = keras.layers.Dropout(params['dense_dropout_rate'])(merged)
    merged = keras.layers.BatchNormalization()(merged)
    res = Dense(1, activation = 'sigmoid')(merged)


    model = keras.Model(inputs=[input1c, input2c, input1, input2], outputs=res)
    model.compile(optimizer= keras.optimizers.Adam(lr = 0.001), loss="binary_crossentropy", 
        metrics=['accuracy'])
    return model

### CNN
def create_model_cnn(params):
    input1 = keras.Input(shape=(MAX_SEQUENCE_LENGTH_WORD, ), dtype='int32')
    input2 = keras.Input(shape=(MAX_SEQUENCE_LENGTH_WORD, ), dtype='int32')
    input1c = keras.Input(shape=(MAX_SEQUENCE_LENGTH_CHAR, ), dtype='int32')
    input2c = keras.Input(shape=(MAX_SEQUENCE_LENGTH_CHAR, ), dtype='int32')
    
    embedder = Embedding(5002, 300, input_length = MAX_SEQUENCE_LENGTH_WORD, 
                         weights = [embedding_matrix_word], trainable = False)#mask_zero=True, 
    v1 = embedder(input1)
    v2 = embedder(input2)
    
#     x1,x2 = get_bilstm_output(embedding_seq1, embedding_seq2, params)
#     rnn_x1 = AttLayer()(x1)
#     rnn_x2 = AttLayer()(x2)

    att1 = Lambda(lambda x: K.max(x,axis = 1))
    
    v11 = get_cnn_output(v1, params)
    v22 = get_cnn_output(v2, params)
    
    v1 = Concatenate(axis=1)([att1(v1),att1(v11)])
    v2 = Concatenate(axis=1)([att1(v2),att1(v22)])
    
    embedder = Embedding(2087, 300, input_length = MAX_SEQUENCE_LENGTH_CHAR, 
                         weights = [embedding_matrix_char], trainable = False)# , mask_zero=True
    att1c = Lambda(lambda x: K.max(x,axis = 1))
    v1c = embedder(input1c)
    v2c = embedder(input2c)
    v11c = get_cnn_output(v1c, params)
    v22c = get_cnn_output(v2c, params)
    v1c = Concatenate(axis=1)([att1c(v1c),att1(v11c)])
    v2c = Concatenate(axis=1)([att1c(v2c),att1(v22c)])
    
    mul = Multiply()([v1,v2])
    sub = Lambda(lambda x: K.abs(x))(Subtract()([v1,v2]))
    maximum = Maximum()([Multiply()([v1,v1]),Multiply()([v2,v2])])
    mulc = Multiply()([v1c,v2c])
    subc = Lambda(lambda x: K.abs(x))(Subtract()([v1c,v2c]))
    maximumc = Maximum()([Multiply()([v1c,v1c]),Multiply()([v2c,v2c])])
    matchlist = Concatenate(axis=1)([mul,sub,mulc,subc,maximum,maximumc])
    matchlist = Dropout(0.2)(matchlist)

    merged = Concatenate(axis=1)([Dense(32,activation = 'relu')(matchlist),
                                     Dense(48,activation = 'sigmoid')(matchlist)])
    
    merged = keras.layers.Dense(params['num_dense'], activation='relu')(merged)
    merged = keras.layers.Dropout(params['dense_dropout_rate'])(merged)
    merged = keras.layers.BatchNormalization()(merged)
    res = Dense(1, activation = 'sigmoid')(merged)


    model = keras.Model(inputs=[input1c, input2c, input1, input2], outputs=res)
    model.compile(optimizer= keras.optimizers.Adam(lr = 0.001), loss="binary_crossentropy", 
        metrics=['accuracy'])
    return model

def unchanged_shape(input_shape):
    return input_shape

def time_distributed(x, layers):
    for l in layers:
        x = TimeDistributed(l)(x)
    return x

def align(input_1, input_2):
    attention = Dot(axes=-1)([input_1, input_2])
    w_att_1 = Lambda(lambda x: softmax(x, axis=1),
                     output_shape=unchanged_shape)(attention)
    w_att_2 = Permute((2,1))(Lambda(lambda x: softmax(x, axis=2),
                             output_shape=unchanged_shape)(attention))
    in1_aligned = Dot(axes=1)([w_att_1, input_1])
    in2_aligned = Dot(axes=1)([w_att_2, input_2])
    return in1_aligned, in2_aligned

def aggregate(x1, x2, x10, x20,  num_class, dense_dim=300, dropout_rate=0.2, activation="relu"):#
    feat1 = concatenate(list(map(lambda l: l(x1), [GlobalAvgPool1D(), GlobalMaxPool1D()])))
    feat2 = concatenate(list(map(lambda l: l(x2), [GlobalAvgPool1D(), GlobalMaxPool1D()])))
    feat10= concatenate(list(map(lambda l: l(x10), [GlobalAvgPool1D(), GlobalMaxPool1D()])))
    feat20 = concatenate(list(map(lambda l: l(x20), [GlobalAvgPool1D(), GlobalMaxPool1D()])))
    x = Concatenate()([feat1, feat2, feat10, feat20]) #
    x = BatchNormalization()(x)
    x = Dense(dense_dim, activation=activation)(x)
    x = Dropout(dropout_rate)(x)
    x = BatchNormalization()(x)
    x = Dense(dense_dim, activation=activation)(x)
    x = Dropout(dropout_rate)(x)
    return x  

def embed_project_atten(q1, q2, encode, projection_dim=300, projection_hidden=200, projection_dropout=0.2,
                           compare_dim=500, compare_dropout=0.2,
                           dense_dim=300, dropout_rate=0.2,
                           lr=1e-3, activation='relu'):
    q1_embed = encode(q1)
    q2_embed = encode(q2)
    
    # Projection
    projection_layers = []
    if projection_hidden > 0:
        projection_layers.extend([
                Dense(projection_hidden, activation=activation),
                Dropout(rate=projection_dropout),
            ])
    projection_layers.extend([
            Dense(projection_dim, activation=None),
            Dropout(rate=projection_dropout),
        ])
    q1_encoded = time_distributed(q1_embed, projection_layers)
    q2_encoded = time_distributed(q2_embed, projection_layers)
    
    # Attention
    q1_aligned, q2_aligned = align(q1_encoded, q2_encoded)    
    
    # Compare
    q1_combined = concatenate([q1_encoded, q2_aligned])
    q2_combined = concatenate([q2_encoded, q1_aligned]) 
    compare_layers = [
        Dense(compare_dim, activation=activation),
        Dropout(compare_dropout),
        Dense(compare_dim, activation=activation),
        Dropout(compare_dropout),
    ]
    q1_compare = time_distributed(q1_combined, compare_layers)
    q2_compare = time_distributed(q2_combined, compare_layers)
    return q1_compare, q2_compare

def lstm_project_atten(q1_embed, q2_embed, projection_dim=300, projection_hidden=200, projection_dropout=0.2,
                           compare_dim=500, compare_dropout=0.2,
                           dense_dim=300, dropout_rate=0.2,
                           lr=1e-3, activation='relu'):
    
    # CuDNN
    lstm0 = CuDNNLSTM(model_params['num_lstm'],
                 return_sequences = True)
    lstm1 = Bidirectional(CuDNNLSTM(model_params['num_lstm']))
    lstm2 = CuDNNLSTM(model_params['num_lstm'])
    
    att1 = Lambda(lambda x: K.max(x,axis = 1))
    q1_embed = lstm0(q1_embed)
    q2_embed = lstm0(q2_embed)
    
    # Projection
    projection_layers = []
    if projection_hidden > 0:
        projection_layers.extend([
                Dense(projection_hidden, activation=activation),
                Dropout(rate=projection_dropout),
            ])
    projection_layers.extend([
            Dense(projection_dim, activation=None),
            Dropout(rate=projection_dropout),
        ])
    q1_encoded = time_distributed(q1_embed, projection_layers)
    q2_encoded = time_distributed(q2_embed, projection_layers)
    
    # Attention
    q1_aligned, q2_aligned = align(q1_encoded, q2_encoded)    
    
    # Compare
    q1_combined = concatenate([q1_encoded, q2_aligned])
    q2_combined = concatenate([q2_encoded, q1_aligned]) 
    compare_layers = [
        Dense(compare_dim, activation=activation),
        Dropout(compare_dropout),
        Dense(compare_dim, activation=activation),
        Dropout(compare_dropout),
    ]
    q1_compare = time_distributed(q1_combined, compare_layers)
    q2_compare = time_distributed(q2_combined, compare_layers)
    return q1_compare, q2_compare

def build_model_combine(params, num_class=1,projection_dim=300, projection_hidden=200, projection_dropout=0.2,
                           compare_dim=500, compare_dropout=0.2,
                           dense_dim=300, dropout_rate=0.2,
                           lr=1e-3, activation='relu'):
    q1 = Input(name='q1',shape=(MAX_SEQUENCE_LENGTH_CHAR,), dtype='int32')
    q2 = Input(name='q2',shape=(MAX_SEQUENCE_LENGTH_CHAR,), dtype='int32')
    q10 = Input(name='q10',shape=(MAX_SEQUENCE_LENGTH_WORD,), dtype='int32')
    q20 = Input(name='q20',shape=(MAX_SEQUENCE_LENGTH_WORD,), dtype='int32')
    
    #  #Embedding Embedding(VOC_SIZE_CHAR, 300, input_length = MAX_SEQUENCE_LENGTH_CHAR)  Embedding(VOC_SIZE_WORD, 300, input_length = MAX_SEQUENCE_LENGTH_WORD) StaticEmbedding(embedding_matrix_char)
    encode = Embedding(VOC_SIZE_CHAR+2, 300, input_length = MAX_SEQUENCE_LENGTH_CHAR) # StaticEmbedding(embedding_matrix_char) #
    
    q1_embed = encode(q1)
    q2_embed = encode(q2)
    
    q1_compare, q2_compare = embed_project_atten(q1, q2, encode)
    
    
    encode = Embedding(VOC_SIZE_WORD+2, 300, input_length = MAX_SEQUENCE_LENGTH_WORD)  # StaticEmbedding(embedding_matrix_word) # 
    q10_embed = encode(q10)
    q20_embed = encode(q20)
    #q10_compare, q20_compare = embed_project_atten(q10_embed, q20_embed, encode)
    
    q1_compare0, q2_compare0 = lstm_project_atten(q1_embed, q2_embed)
    
    q10_compare0, q20_compare0 = lstm_project_atten(q10_embed, q20_embed)
    
    
    q1_compare0 = concatenate([q1_compare0, q1_embed])
    q2_compare0 = concatenate([q2_compare0, q2_embed])
    q10_compare0 = concatenate([q10_compare0, q10_embed])
    q20_compare0 = concatenate([q20_compare0, q20_embed])
    # import pdb; pdb.set_trace()
    q1_compare1, q2_compare1 = lstm_project_atten(q1_compare0, q2_compare0)
    
    q10_compare1, q20_compare1 = lstm_project_atten(q10_compare0, q20_compare0)
    
    
    # Aggregate
    x0 = aggregate(q1_compare1, q2_compare1, q10_compare1, q20_compare1, num_class)#
    x1 = aggregate(q1_embed, q2_embed, q10_embed, q20_embed, num_class)#
    x = concatenate([x0, x1])
    scores = Dense(num_class, activation='sigmoid')(x)
    
    model = Model(inputs=[q1, q2, q10, q20], outputs=scores)#
    return model

def build_model(params, num_class=1,projection_dim=300, projection_hidden=200, projection_dropout=0.2,
                           compare_dim=500, compare_dropout=0.2,
                           dense_dim=300, dropout_rate=0.2,
                           lr=1e-3, activation='relu'):
    q1 = Input(name='q1',shape=(MAX_SEQUENCE_LENGTH_CHAR,), dtype='int32')
    q2 = Input(name='q2',shape=(MAX_SEQUENCE_LENGTH_CHAR,), dtype='int32')
    q10 = Input(name='q10',shape=(MAX_SEQUENCE_LENGTH_WORD,), dtype='int32')
    q20 = Input(name='q20',shape=(MAX_SEQUENCE_LENGTH_WORD,), dtype='int32')
    
    #  #Embedding Embedding(VOC_SIZE_CHAR, 300, input_length = MAX_SEQUENCE_LENGTH_CHAR)  Embedding(VOC_SIZE_WORD, 300, input_length = MAX_SEQUENCE_LENGTH_WORD) StaticEmbedding(embedding_matrix_char)
    encode = Embedding(VOC_SIZE_CHAR+2, 300, input_length = MAX_SEQUENCE_LENGTH_CHAR) # StaticEmbedding(embedding_matrix_char) #
    q1_compare, q2_compare = embed_project_atten(q1, q2, encode)
    
    
    encode = Embedding(VOC_SIZE_WORD+2, 300, input_length = MAX_SEQUENCE_LENGTH_WORD)  # StaticEmbedding(embedding_matrix_word) # 
    q10_compare, q20_compare = embed_project_atten(q10, q20, encode)
    
    
    # Aggregate
    scores = aggregate(q1_compare, q2_compare, q10_compare, q20_compare, num_class)#
    
    scores = Dense(num_class, activation='sigmoid')(scores)
    model = Model(inputs=[q1, q2, q10, q20], outputs=scores)#
    return model

In [52]:
get_embedding_output_vec

<function __main__.get_embedding_output_vec>

In [53]:
len_dict = defaultdict(int)
for i in range(train_data.shape[0]):
    s1 = str(train_data.iloc[i, 1]).split(' ')
    len_dict[len(s1)] += 1
    s1 = str(train_data.iloc[i, 2]).split(' ')
    len_dict[len(s1)] += 1
print len_dict

defaultdict(<type 'int'>, {1: 19, 2: 539, 3: 6450, 4: 16541, 5: 28024, 6: 32512, 7: 29288, 8: 22182, 9: 15893, 10: 11586, 11: 8220, 12: 6044, 13: 4422, 14: 3369, 15: 2625, 16: 1995, 17: 1599, 18: 1247, 19: 1006, 20: 757, 21: 643, 22: 499, 23: 416, 24: 337, 25: 251, 26: 232, 27: 211, 28: 169, 29: 107, 30: 110, 31: 75, 32: 69, 33: 75, 34: 49, 35: 47, 36: 42, 37: 33, 38: 36, 39: 42, 40: 21, 41: 15, 42: 17, 43: 17, 44: 10, 45: 12, 46: 11, 47: 16, 48: 4, 49: 16, 50: 9, 51: 7, 52: 7, 53: 5, 54: 5, 55: 3, 56: 3, 58: 2, 59: 1, 60: 2, 61: 1, 62: 1, 64: 3, 65: 1, 66: 1, 90: 1})


In [54]:
word_squence_ques1_char = list(train_data_char.iloc[:, 1])
word_squence_ques1_char = [[int(im) for im in item.split(' ')] for item in word_squence_ques1_char]
word_squence_ques2_char = list(train_data_char.iloc[:, 2])
word_squence_ques2_char = [[int(im) for im in item.split(' ')] for item in word_squence_ques2_char]

word_squence_ques1_word = list(train_data_word.iloc[:, 1])
word_squence_ques1_word = [[int(im) for im in item.split(' ')] for item in word_squence_ques1_word]
word_squence_ques2_word = list(train_data_word.iloc[:, 2])
word_squence_ques2_word = [[int(im) for im in item.split(' ')] for item in word_squence_ques2_word]

MAX_SEQUENCE_LENGTH_WORD = 50 # char 40 word 30
MAX_SEQUENCE_LENGTH_CHAR = 60 # char 40 word 30
word_squence_ques1_char_len = [len(item) for item in word_squence_ques1_char]
word_squence_ques2_char_len = [len(item) for item in word_squence_ques2_char]
word_squence_ques1_word_len = [len(item) for item in word_squence_ques1_word]
word_squence_ques2_word_len = [len(item) for item in word_squence_ques2_word]
word_squence_ques1_char = pad_sequences(word_squence_ques1_char, maxlen=MAX_SEQUENCE_LENGTH_CHAR)
word_squence_ques2_char = pad_sequences(word_squence_ques2_char, maxlen=MAX_SEQUENCE_LENGTH_CHAR)
word_squence_ques1_word = pad_sequences(word_squence_ques1_word, maxlen=MAX_SEQUENCE_LENGTH_WORD)
word_squence_ques2_word = pad_sequences(word_squence_ques2_word, maxlen=MAX_SEQUENCE_LENGTH_WORD)

In [57]:
class Metrics(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.val_f1s = []

    def on_epoch_end(self, epoch, logs={}):
        val_predict = (np.asarray(self.model.predict(self.validation_data[:-3])))#
        val_predict = [1 if item > 0.63 else 0 for item in val_predict]
        val_targ = self.validation_data[-3]
        _val_f1 = f1_score(val_targ, val_predict)
        self.val_f1s.append(_val_f1)
        print "epoch end, f1-score: ", _val_f1
metrics = Metrics()

from sklearn.model_selection import StratifiedKFold
from keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau
X_train_q1 = word_squence_ques1_char
X_train_q2 = word_squence_ques2_char
X_train_q10 = word_squence_ques1_word
X_train_q20 = word_squence_ques2_word
X_train_magic = all_feas
y_train = train_data.iloc[:, 3]
word_squence_ques1_char_len = np.array(word_squence_ques1_char_len).reshape(-1, 1)
word_squence_ques2_char_len = np.array(word_squence_ques2_char_len).reshape(-1, 1)
word_squence_ques1_word_len = np.array(word_squence_ques1_word_len).reshape(-1, 1)
word_squence_ques2_word_len = np.array(word_squence_ques2_word_len).reshape(-1, 1)
model_checkpoint_path = '../data/checkpoints/stacking-word-word-mix-checkpoint.h5'

class_weight = {0: 1.,
                1: 5.,}

X_fold_val_q1_out,X_fold_val_q2_out, X_fold_val_magic_out = [0, 0, 0]

def model_train(model_func, X_train_q1, X_train_q2, X_train_q10, X_train_q20, X_train_magic, y_train, model_checkpoint_path):
    NUM_FOLDS = 5
    kfold = StratifiedKFold(
        n_splits=NUM_FOLDS,
        shuffle=True,
        random_state=30
    )
    BATCH_SIZE = 1024
    MAX_EPOCHS = 100

    for fold_num, (ix_train, ix_val) in enumerate(kfold.split(X_train_q1, y_train)):

        # Augment the training set by mirroring the pairs.
        X_fold_train_q1 = np.vstack([X_train_q1[ix_train], X_train_q2[ix_train]])
        X_fold_train_q2 = np.vstack([X_train_q2[ix_train], X_train_q1[ix_train]])
        X_fold_train_q10 = np.vstack([X_train_q10[ix_train], X_train_q20[ix_train]])
        X_fold_train_q20 = np.vstack([X_train_q20[ix_train], X_train_q10[ix_train]])
        X_fold_train_char0_len = np.vstack([word_squence_ques1_char_len[ix_train], word_squence_ques2_char_len[ix_train]])
        X_fold_train_char1_len = np.vstack([word_squence_ques2_char_len[ix_train], word_squence_ques1_char_len[ix_train]])
        X_fold_train_word0_len = np.vstack([word_squence_ques1_word_len[ix_train], word_squence_ques2_word_len[ix_train]])
        X_fold_train_word1_len = np.vstack([word_squence_ques2_word_len[ix_train], word_squence_ques1_word_len[ix_train]])
        X_fold_train_magic = np.vstack([X_train_magic[ix_train], X_train_magic[ix_train]])
        print X_fold_train_char0_len.shape
        print X_fold_train_char1_len.shape
        print X_fold_train_word1_len.shape

        X_fold_val_q1 = np.vstack([X_train_q1[ix_val], X_train_q2[ix_val]])
        X_fold_val_q2 = np.vstack([X_train_q2[ix_val], X_train_q1[ix_val]])
        X_fold_val_q10 = np.vstack([X_train_q10[ix_val], X_train_q20[ix_val]])
        X_fold_val_q20 = np.vstack([X_train_q20[ix_val], X_train_q10[ix_val]])
        X_fold_val_char0_len = np.vstack([word_squence_ques1_char_len[ix_val], word_squence_ques2_char_len[ix_val]])
        X_fold_val_char1_len = np.vstack([word_squence_ques2_char_len[ix_val], word_squence_ques1_char_len[ix_val]])
        X_fold_val_word0_len = np.vstack([word_squence_ques1_word_len[ix_val], word_squence_ques2_word_len[ix_val]])
        X_fold_val_word1_len = np.vstack([word_squence_ques2_word_len[ix_val], word_squence_ques1_word_len[ix_val]])
        X_fold_val_magic = np.vstack([X_train_magic[ix_val], X_train_magic[ix_val]])
        X_fold_train_magic[np.isnan(X_fold_train_magic)] = 0
        
        
        # Ground truth should also be "mirrored".
        y_fold_train = np.concatenate([y_train[ix_train], y_train[ix_train]])
        y_fold_val = np.concatenate([y_train[ix_val], y_train[ix_val]])

        print('Fitting fold')

        # Compile a new model.
        # model1 = create_model_cnn(model_params)
        model = model_func(model_params)

        model.compile(optimizer= keras.optimizers.Adam(lr = 0.001), loss="binary_crossentropy", 
            metrics=['accuracy'])
        # Train.
        model.fit(
            [X_fold_train_q1, X_fold_train_q2, 
             X_fold_train_q10, X_fold_train_q20, 
#              X_fold_train_char0_len, X_fold_train_char1_len,
#              X_fold_train_word0_len, X_fold_train_word1_len,
             # X_fold_train_magic
            ], y_fold_train,#
            validation_data=([X_fold_val_q1, X_fold_val_q2, 
                              X_fold_val_q10, X_fold_val_q20, 
#                               X_fold_val_char0_len, X_fold_val_char1_len,
#                               X_fold_val_word0_len,X_fold_val_word1_len,
                              # X_fold_val_magic
                             ], y_fold_val),#

            batch_size=BATCH_SIZE,
            epochs=MAX_EPOCHS,
            verbose=1,
            shuffle=True,
            class_weight=class_weight,

            callbacks=[
                # Stop training when the validation loss stops improving.
                EarlyStopping(
                    monitor='val_loss',
                    min_delta=0.001,
                    patience=20,
                    verbose=1,
                    mode='auto',
                ),
                ReduceLROnPlateau(
                    monitor='val_loss', 
                    factor=0.1, 
                    patience=5, 
                    min_delta=0.0001,
                    cooldown=1, 
                ),
                # Save the weights of the best epoch.
                ModelCheckpoint(
                    os.path.join(model_checkpoint_path + str(fold_num) + 'weights.{epoch:02d}.hdf5'),
                    monitor='val_loss',
                    save_best_only=False,
                    verbose=2,
                    mode='auto',
                ),
                metrics
            ],
        )
        break
    
    # Clear GPU memory.
    K.clear_session()
    del X_fold_train_q1, X_fold_train_q2, X_fold_train_magic
    del X_fold_val_q1, X_fold_val_q2, X_fold_val_magic
    del model
    gc.collect()
    # return score

model_path = '../data/checkpoints/test/'
# model_train(create_model, X_train_q1, X_train_q2, X_train_magic, y_train, model_path)


model_params = {
    'dense_dropout_rate': 0.3,
    'lstm_dropout_rate': 0.3,
    'num_dense': 150,
    'num_lstm': 128,
    'num_filters':32
}

model_train(build_model_combine, X_train_q1, X_train_q2, X_train_q10, X_train_q20, X_train_magic, y_train, model_path)


(158360, 1)
(158360, 1)
(158360, 1)
Fitting fold
Train on 158360 samples, validate on 39592 samples
Epoch 1/100


ResourceExhaustedError: OOM when allocating tensor with shape[51200,500] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: time_distributed_471/MatMul = MatMul[T=DT_FLOAT, _class=["loc:@training_1/Adam/gradients/time_distributed_471/MatMul_grad/MatMul"], transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/device:GPU:0"](time_distributed_470/cond/Merge, time_distributed_467/kernel/read)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[Node: loss_1/mul/_3535 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_11171_loss_1/mul", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]
Hint: If you want to see a list of allocated tensors whe

[<tf.Tensor 'input_33_1:0' shape=(?, 40) dtype=int32>,
 <tf.Tensor 'input_34_1:0' shape=(?, 40) dtype=int32>,
 <tf.Tensor 'input_31_1:0' shape=(?, 30) dtype=int32>,
 <tf.Tensor 'input_32_1:0' shape=(?, 30) dtype=int32>]

In [29]:
MAX_SEQUENCE_LENGTH_CHAR

60

In [30]:
MAX_SEQUENCE_LENGTH_WORD

50

In [ ]:
model2 = keras.models.load_model('../data/checkpoints/test/0weights.01.hdf5', custom_objects={'AttLayer':AttLayer})
model2.inputs